# Code to do estimations of resources for AWS
- Corresponding to experiment 2: PoC: Groundstate of LiH with ActiveSpaceOptimization on sv

In [1]:
from braket.tracking import Tracker
t = Tracker().start()

In [2]:
from pprint import pprint

import matplotlib.pyplot as plt
from braket.aws import AwsDevice
from braket.devices import Devices
from braket.jobs import hybrid_job, save_job_result
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import BackendEstimator
from qiskit_algorithms.optimizers import SLSQP

from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
from qiskit_nature.second_q.transformers import FreezeCoreTransformer
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock

from qiskit_nature.second_q.drivers import PySCFDriver

from qiskit_aer.primitives import Estimator as AerEstimator

from qiskit_braket_provider import BraketProvider, BraketLocalBackend, to_braket
import time
from datetime import datetime

import numpy as np

In [3]:
NQUBITS = 2
MAXITER = 1000
SHOTS = 1000
DIST = 1.6

## Normal with Jordan-Wigner and TwoQubitReduction

In [4]:
def get_qubit_op(dist):
    # Define Molecule
    molecule = MoleculeInfo(
        # Coordinates in Angstrom
        symbols=["Li", "H"],
        coords=([0.0, 0.0, 0.0], [dist, 0.0, 0.0]),
        multiplicity=1,  # = 2*spin + 1
        charge=0,
    )

    driver = PySCFDriver.from_molecule(molecule)
    problem = driver.run()

    mapper = ParityMapper(num_particles=problem.num_particles)
    qubit_op = mapper.map(problem.second_q_ops()[0])
    print (f"Problem spatial orbitals {problem.num_spatial_orbitals}")
    print (f"Problem particles {problem.num_particles}")
    return qubit_op, problem.num_particles, problem.num_spatial_orbitals, problem, mapper

In [5]:
def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

## Braket SV1

In [6]:
backend = BraketProvider().get_backend("SV1")

/tmp/ipykernel_4827/3960545136.py:1: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = BraketProvider().get_backend("SV1")
/tmp/ipykernel_4827/3960545136.py:1: DeprecationWarning: The method ``qiskit.providers.provider.ProviderV1.get_backend()`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will  be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = BraketProvider().get_backend("SV1")


In [7]:
#backend = BraketLocalBackend()

In [8]:
estimator = BackendEstimator(backend=backend, options={"shots": SHOTS})

In [9]:
slsqp = SLSQP(maxiter=MAXITER)

In [10]:
distances = [DIST] #np.arange(1.5, 3.0, 0.1)
vqe_energies = []
optimizer = SLSQP(maxiter=MAXITER)

start = time.time()
# pylint: disable=undefined-loop-variable,line-too-long
for dist in distances:
    (qubit_op, num_particles, num_spatial_orbitals, problem, mapper) = get_qubit_op(
        dist
    )
    init_state = HartreeFock(num_spatial_orbitals, num_particles, mapper)
    var_form = UCCSD(
        num_spatial_orbitals, num_particles, mapper, initial_state=init_state
    )
    print(f"Qubits needed in ansatz: {var_form.num_qubits}")
    vqe = VQE(
        estimator,
        var_form,
        slsqp,
        initial_point=[0] * var_form.num_parameters,
    )
    vqe_calc = vqe.compute_minimum_eigenvalue(qubit_op)
    vqe_result = problem.interpret(vqe_calc).total_energies[0].real
    vqe_energies.append(vqe_result)
    print(
        f"Interatomic Distance: {np.round(dist, 2)}",
        f"VQE Result: {vqe_result:.5f}",
        f"Cost function evals: {vqe_calc.cost_function_evals}",
        f"Optimizer evals: {vqe_calc.optimizer_evals}",
    )
    
end = time.time()
# print execution time
print('Code execution time [sec]:', end - start)

Problem spatial orbitals 6
Problem particles (2, 2)
Qubits needed in ansatz: 10


/home/marc/.local/lib/python3.10/site-packages/qiskit_braket_provider/providers/adapter.py:489: UserWarning: Device does not support global phase; global phase of 1.5707963267948966 will not be included in Braket circuit
  warnings.warn(


AlgorithmError: 'The primitive job to evaluate the energy failed!'

In [11]:
print("Quantum Task Summary")
print(t.quantum_tasks_statistics())
print(f"Estimated cost to run this example: {t.qpu_tasks_cost() + t.simulator_tasks_cost()} USD")

Quantum Task Summary
{'arn:aws:braket:::device/quantum-simulator/amazon/sv1': {'shots': 114000, 'tasks': {'COMPLETED': 114}}}
Estimated cost to run this example: 0 USD
